In [1]:
import sys
import os
import logging
import pandas as pd
import numpy as np
import json
from fmlc import eFMU

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
# Append src directory to import forecaster library files
sys.path.append('../src')

import fcSelector
import fcLib

In [3]:
class ForecasterWrapper(eFMU):
    def __init__(self):
        self.input = {
            'training-data':None, 
            'fcList':None, 
            'input-data': None
        }      
        
        self.output = {
            'model-summary': None,
            'output-data': None,
        }
        
        # initialize selector and forecaster attributes
        self.framework = None
        self.bestModel = None
        self.bestScore = None
        self.bestModelName = None
        
        # initialize selected forecaster prediction
        self.fcPrediction = None
        
        self.msg = None
       
    def compute(self):
        
        # initialize msg to return
        self.msg = None
        self.newPrediction = False

        # initialize forecastor selection framwork
        self.framework = fcSelector.ForecasterFramework(data=self.input['training-data'], fcList=self.input['fcList'])

        # if framework selection has not been made, run evaluate method
        if self.bestModel is None:
        
            # if model hasn't been selected yet, run evaluate method
            try:
                self.framework.evaluateAll()

                # unpack framework attributes to wrapper
                self.bestModel = self.framework.bestModel
                self.bestScore = self.framework.bestScore
                self.bestModelName = self.framework.bestModelName
             
            except Exception as e:
                #logging.error(e)
                self.msg = 'ERROR: Failed to train forecasters in fc list' + str(e)
            
        # run prediction on fcData if passed
        if self.input['input-data'] is not None:
            try:
                # run prediction method of best 
                self.fcPrediction = self.bestModel.predict(self.input['input-data'])
                
                self.newPrediction = True

            except Exception as e:
                #logging.error(e)
                self.msg = 'ERROR: Failed to generate new prediction' + str(e)
    
        # package results into self.output
        self.output['model-summary'] = {
            'bestModelName': self.bestModelName,
            'bestScore': self.bestScore
        }
        
        if self.newPrediction:
            self.output['output-data'] = self.fcPrediction.tolist() #json.dumps(self.fcPrediction.__dict__)
            
        # data management wrapper or data mgmt in this class

        # if no msg has been define, default to Done
        if self.msg is None:
             self.msg = 'Done.'
        
        # Return status message
        return self.msg


#### Test FC Selector Wrapper

In [4]:
cwd = os.getcwd()
file = '../resources/data/sample_data.csv'
path = os.path.join(cwd, file)

data = pd.read_csv(path)
data = data.rename(columns={'OAT': 'TDryBul'})

data['occ mode'] = np.where(data['Power (kW)'] == 1, 'unocc', 'occ')

X = data[['Time of Week (hr)', 'TDryBul (deg C)']]
y = data['Power (kW)']

# package data for framework
trainData = {
    'X': X,
    'y': y
}

# define an example fcList: forecaster names and parameters
fcList = [
    {
        'fun': fcLib.forecasters['random forest pipeline']['fun'],
        'parameter': fcLib.forecasters['random forest pipeline']['parameter']
    },
    {
        'fun': fcLib.forecasters['sarimax']['fun'],
        'parameter': fcLib.forecasters['sarimax']['parameter']
    },
    {
        'fun': fcLib.forecasters['tuned-Total mlp']['fun'],
        'parameter': fcLib.forecasters['tuned-Total mlp']['parameter']
    },
    {
        'fun': fcLib.forecasters['tuned-Total mlp']['fun'],
        'parameter': fcLib.forecasters['tuned-Total mlp']['parameter']
    },
]

input = {
    'training-data':trainData, 
    'fcList': fcList, 
    'input-data': X
}

# instantiate forecast framework wrapper
newWrapper = ForecasterWrapper()

# pass inputs
newWrapper.input = input

# run compute method to train and predict
newWrapper.compute()

"ERROR: Failed to generate new prediction'NoneType' object has no attribute 'predict'"

In [5]:
newWrapper.output['fcPrediction']

KeyError: 'fcPrediction'